## This notebook is used for testing MTAOS communication with the AOS components: M1M3, M2, and hexapods

Step-by-step description of what we do can be found in 
Jira test case: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T???? 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

In [ ]:
from lsst.ts import salobj
import asyncio
import os

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

from lsst.ts.idl.enums import MTM1M3

from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u

from aosTools import *

plt.jet();

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ["LSST_DDS_DOMAIN_ID"])

In [ ]:
#index is an integter which helps avoid multple users starting same controller
script = salobj.Controller("Script", index=7)

In [ ]:
mtcs = MTCS(script.domain)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

In [ ]:
mtcs.components_attr

In [ ]:
camhex = mtcs.rem.mthexapod_1
m2hex = mtcs.rem.mthexapod_2
m1m3 = mtcs.rem.mtm1m3
m2 = mtcs.rem.mtm2
aos = mtcs.rem.mtaos

In [ ]:
def printLogMessage(data):
    print(f"{data.level}: {data.message}")
aos.evt_logMessage.callback = printLogMessage
await aos.cmd_setLogLevel.set_start(level=10, timeout=5)

In [ ]:
await aos.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await salobj.set_summary_state(aos, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(aos, salobj.State.ENABLED) #, settingsToApply='default') #leave this out!!

#### Check summary state of each CSC

In [ ]:
await checkAOSSummaryStates(aos, m1m3, m2, camhex, m2hex)

### Get M1M3 Ready: raise mirorr, turn on FB, clear forces

In [ ]:
print('Re-enabling M1M3')
await salobj.set_summary_state(m1m3, salobj.State.STANDBY) 
await salobj.set_summary_state(m1m3, salobj.State.ENABLED, settingsToApply = 'Default') #enable m1m3

In [ ]:
await readyM1M3(m1m3)

In [ ]:
#%matplotlib inline
await plotM1M3Forces(m1m3)

### Get M2 Ready: turn on FB, clear forces

In [ ]:
await salobj.set_summary_state(m2, salobj.State.STANDBY) 
#await salobj.set_summary_state(m1m3, salobj.State.ENABLED, settingsToApply = 'Default') #enable m1m3

In [ ]:
await readyM2(m2)

In [ ]:
await plotM2Forces(m2)

### Get cam hex Ready: check config; make sure LUT is on, and has valid inputs; make sure hex is at LUT position

In [ ]:
await readyHexForAOS(camhex)

In [ ]:
lutMode = await camhex.evt_compensationMode.aget(timeout=10)
if not lutMode.enabled:
    await camhex.cmd_setCompensationMode.set_start(enable=1, timeout=10)
print("compsensation mode enabled?",lutMode.enabled, pd.to_datetime(lutMode.private_sndStamp, unit='s'))
print("Does the hexapod has enough inputs to do LUT compensation?")
#Note: the target events are what the hex CSC checks; if one is missing, the entire LUT will not be applied
a = camhex.evt_compensationOffset.get()
print('mount elevation = ', a.elevation)
print('mount azimth = ', a.azimuth)
print('rotator angle = ', a.rotation)
print('? temperature = ', a.temperature)
print('x,y,z,u,v,w = ', a.x, a.y, a.z, a.u, a.v, a.w)

In [ ]:
posU = await camhex.evt_uncompensatedPosition.aget(timeout=10.)

In [ ]:
abs(max([getattr(posU, i) for i in 'xyzuvw']))

In [ ]:
await printHexUncompensatedAndCompensated(camhex)

In [ ]:
await camhex.cmd_move.set_start(x=0,y=0,z=100, u=0,v=0,w=0,sync=True) #to avoid the timeout if it is already at 0.
await asyncio.sleep(5.)
await moveHexTo0(camhex)

In [ ]:
await printHexUncompensatedAndCompensated(camhex)

In [ ]:
mountAngle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mountAngle.actualPosition)
elev = mountAngle.actualPosition

### Get M2 hex Ready: check config; make sure LUT is on, and has valid inputs; make sure hex is at LUT position

In [ ]:
await readyHexForAOS(m2hex)

In [ ]:
await printHexUncompensatedAndCompensated(m2hex)

In [ ]:
#await m2hex.cmd_move.set_start(x=0,y=0,z=100, u=0,v=0,w=0,sync=True) #to avoid the timeout if it is already at 0.
#await asyncio.sleep(5.)
await moveHexTo0(m2hex)

In [ ]:
await printHexUncompensatedAndCompensated(m2hex)

### Step 1: add aberrations via OFC

In [ ]:
zernikes = np.zeros(19)
zerIdx = np.arange(4, 23)
zernikes[7-4] = 1 #add 1um of z7
print(zerIdx)
print(zernikes)

In [ ]:
await aos.cmd_addAberration.set_start(wf = zernikes) #config=??

In [ ]:
def compSentApplied(aos, m1m3, m2, camhex, m2hex):
    dof = aos.evt_degreeOfFreedom.get()
    m1m3C = aos.evt_m1m3Correction.get()
    m2C = aos.evt_m2Correction.get()
    camhexC = aos.evt_cameraHexapodCorrection.get()
    m2hexC = aos.evt_m2HexapodCorrection.get()
    print(pd.to_datetime(dof.private_sndStamp, unit='s'))
    fig, ax = plt.subplots(2,3, figsize=(19,8) )

    aa = np.array(dof.aggregatedDoF)
    vv = np.array(dof.visitDoF)

    ##--------------------------------------
    ax[0][0].plot(aa[:10],'-bo', label='aggregatedDoF')
    ax[0][0].plot(vv[:10],'-rx', label='visitDoF')
    ax[0][0].set_title('hexapod DoF')
    ax[0][0].legend()

    ax[0][1].plot(aa[10:], '-bo', label='aggregatedDoF')
    ax[0][1].plot(vv[10:],'-rx', label='visitDoF')
    ax[0][1].set_title('Mirrors DoF')
    ax[0][1].legend()

    ##--------------------------------------
    aa = m1m3.evt_appliedActiveOpticForces.get()
    ax[0][2].plot(m1m3C.zForces,'-o', label='forces sent')
    ax[0][2].plot(aa.zForces, '-rx', label='forces applied')
    ax[0][2].set_title('M1M3 Forces')
    ax[0][2].legend()

    aa = m2.tel_axialForce.get()
    ax[1][0].plot(m2C.zForces,'-o', label='forces sent')
    ax[1][0].plot(aa.applied, '-x', label='forces applied')
    ax[1][0].set_title('M2 Forces')
    ax[1][0].legend()

    ##--------------------------------------
    aa = np.array([getattr(camhexC,i) for i in ['x', 'y', 'z', 'u','v','w']])
    aam2 = np.array([getattr(m2hexC,i) for i in ['x', 'y', 'z', 'u','v','w']])
    uu = np.array([getattr(camhex.evt_uncompensatedPosition.get(),i) for i in ['x','y','z', 'u','v','w']])
    uum2 = np.array([getattr(m2hex.evt_uncompensatedPosition.get(),i) for i in ['u','v','z', 'u','v','w']])

    ax[1][1].plot(aa[:3], '-ro', label='cam hex xyz Sent', markersize=8)
    ax[1][1].plot(aam2[:3],'-bx', label='m2 hex xyz Sent')
    ax[1][1].plot(uu[:3], '-o',  label='cam hex xyz Applied')
    ax[1][1].plot(aam2[:3], '-v', label='m2 hex xyz Applied')
    ax[1][1].set_title('Hex xyz')
    ax[1][1].legend()

    ax[1][2].plot(aa[3:], '-ro', label='cam hex uvw Sent')
    ax[1][2].plot(aam2[3:], '-bx', label='m2 hex uvw Sent')
    ax[1][2].plot(uu[3:], '-o', label='cam hex uvw Applied')
    ax[1][2].plot(uum2[3:], '-v', label='m2 hex uvw Applied')
    ax[1][2].set_title('M2 Hex xyzuvw')
    ax[1][2].legend()
    
    return dof, aa, aam2, uu, uum2    

dof, camsent, m2sent, camrvd, m2rvd =  compSentApplied(aos, m1m3, m2, camhex, m2hex)

### Step 2: add same aberrations twice. Are the aggregateDOF accumulating?

In [ ]:
await aos.cmd_addAberration.set_start(wf = zernikes) #config=??

In [ ]:
dof2, camsent2, m2sent2, camrvd2, m2rvd2 = compSentApplied(aos, m1m3, m2, camhex, m2hex)

This depends on the control strategy we are using. For x00, do not expect 2 or 1.

In [ ]:
camsent2[:5]/camsent[:5]

In [ ]:
m2sent2[:5]/m2sent[:5]

In [ ]:
plt.plot(np.array(dof2.aggregatedDoF[:10])/np.array(dof.aggregatedDoF[:10]))

In [ ]:
plt.plot(np.array(dof2.visitDoF[:10])/np.array(dof.visitDoF[:10]))

### Step 3: Reset the corrections

In [ ]:
await aos.cmd_resetCorrection.set_start()

In [ ]:
##the following should be done by cmd_resetCorrection, but right now this is not the case
await aos.cmd_issueCorrection.set_start() 
#issue the resetted correction, which should be equivalent to the below
#await m1m3.cmd_applyActiveOpticForces.set_start(zForces=[0]*156)
#await m2.cmd_resetForceOffsets.set_start()
#await camhex.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)
#await m2hex.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [ ]:
plt.plot(aos.evt_m1m3Correction.get().zForces)
plt.plot(m1m3.evt_appliedActiveOpticForces.get().zForces);

### Step 4: Double the aberration, do the corrections double? yes

In [ ]:
zernikes[7-4] = 2 # z7 = 2um

In [ ]:
await aos.cmd_addAberration.set_start(wf = zernikes) #config=??

In [ ]:
dof3, camsent3, m2sent3, camrvd3, m2rvd3 = compSentApplied(aos, m1m3, m2, camhex, m2hex)

In [ ]:
camsent3[:5]/camsent[:5]

In [ ]:
m2sent3[:5]/m2sent[:5]

### Step 5: Observe the corrections being rejected

In [ ]:
await lowerM1M3(m1m3)

In [ ]:
#save the forces and positions before attempting adding aberrations
m1m3F0 = m1m3.evt_appliedActiveOpticForces.get().zForces
m2F0 = m2.tel_axialForce.get().applied
camhexP0 = np.array([getattr(camhex.evt_uncompensatedPosition.get(),i) for i in ['x','y','z', 'u','v','w']])
m2hexP0 = np.array([getattr(m2hex.evt_uncompensatedPosition.get(),i) for i in ['u','v','z', 'u','v','w']])

In [ ]:
#save MTAOS corrections before attempting adding aberrations
dofa0 = aos.evt_degreeOfFreedom.get().aggregatedDoF
dofv0 = aos.evt_degreeOfFreedom.get().visitDoF
m1m3C0 = aos.evt_m1m3Correction.get().zForces
m2C0 = aos.evt_m2Correction.get().zForces
camhexC0 = np.array([getattr(aos.evt_cameraHexapodCorrection.get(),i) for i in ['x','y','z', 'u','v','w']])
m2hexC0 = np.array([getattr(aos.evt_m2HexapodCorrection.get(),i) for i in ['x','y','z', 'u','v','w']])

In [ ]:
plt.plot(m1m3C0); #this should be same as plot above (a few cells back). We simply just saved the data

In [ ]:
#Expect this command to be rejected, because M1M3 is parked
zernikes[7-4] = 1
await aos.cmd_addAberration.set_start(wf = zernikes) #config=??

#### the addAberration command is supposed to have undone corrections to M2, camhex, and m2hex. Let's check that

In [ ]:
#save the forces and positions before attempting adding aberrations
m1m3F1 = m1m3.evt_appliedActiveOpticForces.get().zForces
m2F1 = m2.tel_axialForce.get().applied
camhexP1 = np.array([getattr(camhex.evt_uncompensatedPosition.get(),i) for i in ['x','y','z', 'u','v','w']])
m2hexP1 = np.array([getattr(m2hex.evt_uncompensatedPosition.get(),i) for i in ['u','v','z', 'u','v','w']])

In [ ]:
print(np.array(m1m3F1)- np.array(m1m3F0))
print(np.array(m2F1)- np.array(m2F0))
print(camhexP1-camhexP0)
print(m2hexP1-m2hexP0)

In [ ]:
plt.plot(m2F1)
plt.plot(m2F0);

In [ ]:
plt.plot(camhexP1)
plt.plot(camhexP0)

In [ ]:
plt.plot(m2hexP1)
plt.plot(m2hexP0)

#### What about MTAOS corrections?

In [ ]:
dofa1 = aos.evt_degreeOfFreedom.get().aggregatedDoF
dofv1 = aos.evt_degreeOfFreedom.get().visitDoF
m1m3C1 = aos.evt_m1m3Correction.get().zForces
m2C1 = aos.evt_m2Correction.get().zForces
camhexC1 = np.array([getattr(aos.evt_cameraHexapodCorrection.get(),i) for i in ['x','y','z', 'u','v','w']])
m2hexC1 = np.array([getattr(aos.evt_m2HexapodCorrection.get(),i) for i in ['x','y','z', 'u','v','w']])

In [ ]:
print(np.array(dofa1)-np.array(dofa0))
print(np.array(dofv1)-np.array(dofv0))
print(np.array(m1m3C1)-np.array(m1m3C0))
print(np.array(m2C1)-np.array(m2C0))
print(np.array(camhexC1)-np.array(camhexC0))
print(np.array(m2hexC1)-np.array(m2hexC0))
#The expectation is that these have changed even when the command got rejected. ??
#We need the rejectCorrection command to MTAOS to the correction from last visit. ??

### rejectCorrection 

This makes sure OFC does proper accounting when commands are rejected.

In [ ]:
await aos.cmd_rejectCorrection.set_start()

In [ ]:
dofa2 = aos.evt_degreeOfFreedom.get().aggregatedDoF
dofv2 = aos.evt_degreeOfFreedom.get().visitDoF
m1m3C2 = aos.evt_m1m3Correction.get().zForces
m2C2 = aos.evt_m2Correction.get().zForces
camhexC2 = np.array([getattr(aos.evt_cameraHexapodCorrection.get(),i) for i in ['x','y','z', 'u','v','w']])
m2hexC2 = np.array([getattr(aos.evt_m2HexapodCorrection.get(),i) for i in ['x','y','z', 'u','v','w']])

In [ ]:
print(np.array(dofa2)-np.array(dofa0))
print(np.array(dofv2)-np.array(dofv0))
print(np.array(m1m3C2)-np.array(m1m3C0))
print(np.array(m2C2)-np.array(m2C0))
print(np.array(camhexC2)-np.array(camhexC0))
print(np.array(m2hexC2)-np.array(m2hexC0))

In [ ]:
## check the forces and hexapod motions

### Close up

In [ ]:
#if we started with enabled state, we need to put it back
await salobj.set_summary_state(m1m3, salobj.State.ENABLED,  settingsToApply='Default')

In [ ]:
await salobj.set_summary_state(mount, salobj.State.ENABLED)

### Check if the telescope is in tracking mode. If yes, need to stop stacking. 
The alternative is to check "MT Mount status" dash board on Chronograf. Make sure there are three "False".

In [ ]:
mountStatus = await mount.evt_axesInPosition.aget(timeout=5.)
rotStatus = await rot.evt_inPosition.aget(timeout=5.)
trackingStatus = mountStatus.elevation and mountStatus.azimuth and rotStatus.inPosition
print('Are we tracking?', trackingStatus)

In [ ]:
await ptg.cmd_stopTracking.set_start(timeout=5.)